In [20]:
import os
import pickle
from utils import get_cache_path
identifier = '3724f21c81d258e672c7a55fc25fadd7_f1084cbc4400de5426071b9e676e0910_8e15dea74a547a17a51aedd3a9ecf505_cab570f625526a1935fcfc2927c2fe16'
alice_enc_hash = 'f1084cbc4400de5426071b9e676e0910'
splits=("train", "val", "test")

toggle = True

if toggle:
    identifier = "73dd8854339a0c92b060783aefd0fdd8_df50a7ac7d8383c54c936ca43fc0edf6_0d291589f92d0e6b54923f2cdf57f062_40255e09e475b0c582af75220b7d74c1"
    alice_enc_hash = "df50a7ac7d8383c54c936ca43fc0edf6"
    

true_path = "data/cache/dataset_3724f21c81d258e672c7a55fc25fadd7_f1084cbc4400de5426071b9e676e0910_8e15dea74a547a17a51aedd3a9ecf505_cab570f625526a1935fcfc2927c2fe16_f1084cbc4400de5426071b9e676e0910.pkl"

cache_path = get_cache_path("data", identifier, alice_enc_hash)
if os.path.exists(cache_path):
    with open(cache_path, 'rb') as f:
        cached = pickle.load(f)
        if isinstance(cached, dict):
            datasets =  {k: cached.get(k) for k in splits}
        else:
            train, val, test = cached
            datasets = {"train": train, "val": val, "test": test}
data_train, data_val, data_test = datasets["train"], datasets["val"], datasets["test"]

10171